In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""This notebook creates the map of TAG in COUNTRY 
for all its editions, provided images have been 
properlycategorized"""

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot import pagegenerators

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot import pagegenerators

In [2]:
import pandas as pd
import numpy as np
from mako.template import Template
from io import StringIO
import random
import seaborn as sns

In [3]:
from geojson import Feature, Point, FeatureCollection
import geojson

In [4]:
from modules.wmtools import coordinate_shaker

In [5]:
YEARS               = [2015, 2016, 2017]

TAG                 = 'WLE'
TAG_EXT             = 'Wiki Loves Earth'
COUNTRY             = "Spain"

BASE_NAME           = "Commons:Wiki Loves in {2}/{1}/{0}"
LOG_PAGES           = ["{0}/Log".format(BASE_NAME.format(YEAR, TAG_EXT, COUNTRY)) for YEAR in YEARS]

BASE_SITE_DB_NAME   = "Commons:Wiki Loves in {1}/{0}".format(TAG_EXT, COUNTRY)
SITE_DB_PAGE        = BASE_SITE_DB_NAME + "/Sites DB"
MAP_WLE_PAGE        = BASE_SITE_DB_NAME + '/Map'

commons_site = pb.Site('commons', 'commons')

In [6]:
annexes = {
   'ES-AN': [u'Anexo:Lugares de importancia comunitaria de Andalucía', 'Andalusia'],
   'ES-AR': [u'Anexo:Lugares de importancia comunitaria de Aragón', 'Aragon'],
   'ES-AS': [u'Anexo:Lugares de importancia comunitaria de Asturias', 'Asturias'],
   'ES-CB': [u'Anexo:Lugares de importancia comunitaria de Cantabria', 'Cantabria'],
   'ES-CM': [u'Anexo:Lugares de importancia comunitaria de Castilla-La Mancha', 'Castile-La Mancha'],
   'ES-CL': [u'Anexo:Lugares de importancia comunitaria de Castilla y León', u'Castile and León'],
   'ES-CT': [u'Anexo:Lugares de importancia comunitaria de Cataluña', 'Catalonia'],
   'ES-MD': [u'Anexo:Lugares de importancia comunitaria de la Comunidad de Madrid', 'Community of Madrid'],
   'ES-VC': [u'Anexo:Lugares de importancia comunitaria de la Comunidad Valenciana', 'Valencian Community'],
   'ES-EX': [u'Anexo:Lugares de importancia comunitaria de Extremadura', 'Extremadura'],
   'ES-IB': [u'Anexo:Lugares de importancia comunitaria de las Islas Baleares', 'Balearic Islands'],
   'ES-CN': [u'Anexo:Lugares de importancia comunitaria de las Islas Canarias', 'Canary Islands'],
   'ES-GA': [u'Anexo:Lugares de importancia comunitaria de Galicia', 'Galicia'],
   'ES-RI': [u'Anexo:Lugares de importancia comunitaria de La Rioja', 'La Rioja'],
   'ES-NC': [u'Anexo:Lugares de importancia comunitaria de Navarra', 'Navarre'],
   'ES-MC': [u'Anexo:Lugares de importancia comunitaria de la Región de Murcia', 'Region of Murcia'],
   'ES-PV': [u'Anexo:Lugares de importancia comunitaria del País Vasco', 'Basque Country'],
   'ES-CE': [u'Anexo:Lugares de importancia comunitaria de Ceuta y Melilla', 'Ceuta'],
   'ES-ML': [u'Anexo:Lugares de importancia comunitaria de Ceuta y Melilla', 'Melilla'],
   'ES-MAGRAMA': [u'Anexo:Lugares de importancia comunitaria del MAGRAMA', 'MAGRAMA']
}

In [7]:
autcom_palette = [i[1:] for i in sns.color_palette('hls', len(annexes)).as_hex()]
autcoms = [annexes[i][1] for i in annexes]
autcom_colors = {autcom: autcom_palette[i] for i, autcom in enumerate(autcoms)}
autcom_colors

{'Andalusia': 'db5f57',
 'Aragon': 'db8657',
 'Asturias': 'dbae57',
 'Balearic Islands': '57d3db',
 'Basque Country': 'c957db',
 'Canary Islands': '57acdb',
 'Cantabria': 'dbd657',
 'Castile and León': '91db57',
 'Castile-La Mancha': 'b9db57',
 'Catalonia': '69db57',
 'Ceuta': 'db57c6',
 'Community of Madrid': '57db6c',
 'Extremadura': '57dbbb',
 'Galicia': '5784db',
 'La Rioja': '575cdb',
 'MAGRAMA': 'db5777',
 'Melilla': 'db579e',
 'Navarre': '7957db',
 'Region of Murcia': 'a157db',
 'Valencian Community': '57db94'}

In [8]:
def to_geojson (row) :
    """For each site of community importance, identified by row['code'], this function
    creates a proper GeoJSON Feature"""
    images_subset_df = images_df[(images_df['code'] == row['code']) & (images_df['width'] > images_df['height'])]
    if len (images_subset_df.index) == 0:
        images_subset_df = images_df[images_df['code'] == row['code']]

    if len(images_subset_df[images_subset_df['qi'] == 'qi']) > 0 :
        popup_image = images_subset_df[images_subset_df['qi'] == 'qi'].sample(1, random_state=0)['image_title'].values[0]
    elif len(images_subset_df[images_subset_df['finalist'] == 'finalist']) > 0 :
        popup_image = images_subset_df[images_subset_df['finalist'] == 'finalist'].sample(1, random_state=0)['image_title'].values[0]
    else :
        popup_image = images_subset_df.sample(1, random_state=0)['image_title'].values[0]

    properties = {"description": "[[File:{0}|150px]]".format(popup_image),
                  "title": "[[:Category:Images of a site of community importance with code {0} from {1} in {3}|{2}]]".format(row['code'], TAG_EXT, row['name'], COUNTRY),
                  "marker-size": "small",
                  "marker-symbol": "circle",
                  "marker-color": autcom_colors[row['aut_com']]}

    feature = Feature(geometry=Point((float(row['longitude']), float(row['latitude']))), 
                      properties=properties
                     )
    return feature

In [9]:
# retrieval of the WLE SCI (site of community importance) log
pb.output('Retrieving --> WLE site of community importance list')
site_list_page = pb.Page(commons_site, SITE_DB_PAGE)
site_list_text = StringIO(site_list_page.text[site_list_page.text.find('\n') + 
                                                      1:site_list_page.text.rfind('\n')])
site_df = pd.read_csv(site_list_text, sep=";",
                      index_col=False,
                      names=["name", "code", "magrama_url", "community",
                            "bio_region", "continent", "min_altitude",
                            "max_altitude", "avg_altitude", "longitude",
                            "latitude", "area", "marine_percentage",
                            "marine_area", "image", "commons_cat", "wikidata_id"])

pb.output('Retrieved --> WLE site of community importance list')

Retrieving --> WLE site of community importance list
Retrieved --> WLE site of community importance list


In [10]:
site_df["aut_com"] = site_df["community"].apply(lambda x: annexes[x][1])

In [11]:
valid_sites = site_df['code'].values
valid_sites

array(['ES0000045', 'ES0000046', 'ES0000047', ..., 'ES7020126',
       'ES7020128', 'ES90ATL01'], dtype=object)

In [12]:
image_columns = ['image_title', 'code', 'uploader', 'uploader_registration',
                 'timestamp', 'date', 'size', 'height', 'width', 'qi', 'finalist']
images_df = pd.DataFrame(columns=image_columns)

In [13]:
# Retrieval of images
pb.output('Retrieving --> {0} in {1} images list from cache'.format(TAG, COUNTRY))
for log_page in LOG_PAGES:
    list_page = pb.Page(commons_site, log_page)
    list_page_text = StringIO(list_page.text[list_page.text.find('\n') + 1:list_page.text.rfind('\n')])
    yearly_df = pd.read_csv(list_page_text,
                            sep=";",
                            index_col=False,
                            names=image_columns
                           ).fillna('')
    images_df = pd.concat([images_df, yearly_df])
pb.output('Retrieved --> {0} in {1} images list from cache'.format(TAG, COUNTRY))

images_df['timestamp'] = pd.to_datetime(images_df['timestamp'], format="%Y-%m-%d %H:%M:%S")

images_df.set_index(["timestamp"], inplace=True)
del images_df.index.name

total_images_length = len(images_df)
total_images_length

Retrieving --> WLE in Spain images list from cache
Retrieved --> WLE in Spain images list from cache


11320

In [14]:
images_extended_df = pd.merge(images_df, site_df, on='code', how='left')
len(images_extended_df.index)

11320

In [15]:
images_per_site = images_extended_df[images_extended_df['code'].isin(valid_sites)]['code'].value_counts()
images_per_site

ES3110005    367
ES0000016    349
ES1110005    158
ES0000046    143
ES2410003    141
ES7020043    135
ES3110002    133
ES4230014    130
ES1200001    127
ES0000035    126
ES0000001    112
ES2420039    111
ES0000023    107
ES3110004     97
ES0000049     95
ES1120005     95
ES4110002     82
ES4160109     81
ES0000149     80
ES5120013     74
ES1110002     74
ES6170003     71
ES0000147     71
ES0000013     70
ES2410017     70
ES0000115     69
ES0000032     68
ES0000020     68
ES0000231     68
ES6140004     67
            ... 
ES4150126      1
ES7020058      1
ES6110007      1
ES4310004      1
ES7020041      1
ES4160122      1
ES7020014      1
ES2430088      1
ES6150005      1
ES5310023      1
ES2430077      1
ES0000135      1
ES2410026      1
ES1200012      1
ES6120001      1
ES7020047      1
ES7020022      1
ES4310027      1
ES6200011      1
ES6150026      1
ES1200016      1
ES5130003      1
ES7020071      1
ES2420124      1
ES7020028      1
ES5223002      1
ES7011003      1
ES1130005     

In [16]:
images_per_site_df = pd.DataFrame(data=images_per_site).reset_index()
images_per_site_df.rename(columns={'index': 'code', 'code': 'count'}, inplace=True)

In [17]:
images_per_site_df = pd.merge(images_per_site_df, site_df, on='code')[['count', 'code', 'name', 'aut_com', 'latitude', 'longitude', 'commons_cat']].fillna('')
images_per_site_df = images_per_site_df.iloc[np.lexsort([images_per_site_df['name'], -images_per_site_df['count']])]
images_per_site_df['name'] = images_per_site_df['name'].map(lambda x: x.replace('_', ' '))
images_per_site_df.head()

,count,code,name,aut_com,latitude,longitude,commons_cat
0,367,ES3110005,Cuenca del río Guadarrama,Community of Madrid,40.502800,-4.005600,Cuenca del Guadarrama (site of community impor...
1,349,ES0000016,Ordesa y Monte Perdido,Aragon,42.640000,-0.031900,Parque Nacional de Ordesa y Monte Perdido
2,158,ES1110005,Costa da Morte,Galicia,42.867300,-9.249500,Costa da Morte
3,143,ES0000046,"Cabo de Gata, Nijar",Andalusia,36.823929,-2.095817,Parque Natural de Cabo de Gata-Níjar
4,141,ES2410003,Los Valles,Aragon,42.810000,-0.680000,Los Valles Occidentales Natural Park


In [18]:
images_per_site_df['geojson'] = images_per_site_df.apply(lambda row: to_geojson(row), axis=1)

In [19]:
features = images_per_site_df['geojson'].tolist()
feature_collection = FeatureCollection(features)
dump = geojson.dumps(feature_collection, ensure_ascii=False, indent=2)
#print(dump)

In [20]:
template = """=== WLE contributions map ===
The map below includes all the contributions, by site of community importance, for all the editions of the contest 
(${years[0]}-${years[-1]}).
<mapframe text="Festivals" latitude="39" longitude="-4" zoom="5" width="800" height="600" align="center"> 
${map}
</mapframe>
"""
vars = {
    "map": dump,
    "years": YEARS
}
t = Template(template)
map_text = t.render(**vars)

In [21]:
maps_page = pb.Page(commons_site, MAP_WLE_PAGE)
maps_page.text = map_text
pb.output('Publishing --> {0} in Spain Statistics'.format(TAG))
maps_page.save("{0} in Spain statistics".format(TAG))

Publishing --> WLE in Spain Statistics
Sleeping for 4.0 seconds, 2018-03-03 15:58:28
Page [[commons:Commons:Wiki Loves in Spain/Wiki Loves Earth/Map]] saved
